<a href="https://colab.research.google.com/github/MariaZharova/test_rec_systems/blob/main/Test_LightFM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import warnings
warnings.filterwarnings("ignore")

import category_encoders as ce
import pandas as pd
#from lightfm import LightFM
#from lightfm.evaluation import precision_at_k
import numpy as np
#from polara.recommender.models import SVDModel, RandomModel
#from polara.recommender.data import RecommenderData
from sklearn.metrics import average_precision_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error, mean_absolute_error
from scipy.sparse import csr_matrix

In [ ]:
data = pd.read_csv('app_dataset_msk.csv')
data.head()

,id,uid_device,action,category,geo_address_id,isactive,price,fv_category,fv_geo,ptn_dadd
0,273726522,00312318-29A0-4E1C-BD78-C0622E14AB66,1,flatSale,1,1,12000000,flatSale,1.0,2022-06-21
1,274140870,00312318-29A0-4E1C-BD78-C0622E14AB66,1,flatSale,1,1,11499000,flatSale,1.0,2022-06-21
2,272703932,004639AD-0CE4-4F63-88CD-BAFA8A5BBC70,1,flatSale,1,1,44500000,flatSale,1.0,2022-06-21
3,274719451,004639AD-0CE4-4F63-88CD-BAFA8A5BBC70,1,flatSale,1,0,15500000,flatSale,1.0,2022-06-21
4,273874137,004639AD-0CE4-4F63-88CD-BAFA8A5BBC70,1,flatSale,1,1,8500000,flatSale,1.0,2022-06-21


In [ ]:
print(data.shape)
data.info()

(226716, 10)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 226716 entries, 0 to 226715
Data columns (total 10 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   id              226716 non-null  int64  
 1   uid_device      226716 non-null  object 
 2   action          226716 non-null  int64  
 3   category        226716 non-null  object 
 4   geo_address_id  226716 non-null  int64  
 5   isactive        226716 non-null  int64  
 6   price           226716 non-null  int64  
 7   fv_category     226715 non-null  object 
 8   fv_geo          226715 non-null  float64
 9   ptn_dadd        226715 non-null  object 
dtypes: float64(1), int64(5), object(4)
memory usage: 17.3+ MB


### LightFM model

In [ ]:
# encoding
ord_encoder = ce.OrdinalEncoder()
data['uid_device_enc'] = ord_encoder.fit_transform(data['uid_device'])['uid_device'] - 1
data['id_enc'] = ord_encoder.fit_transform(data['id'].apply(str))['id'] - 1

# create train and test
print(data.shape)
data_train = data.iloc[:data.shape[0]//3, :]
data_test = data.iloc[data.shape[0]//3:, :]

# create sparse matrices
csr_user_item_train = csr_matrix((data_train['action'], (data_train['uid_device_enc'], data_train['id_enc'])))
#csr_user_item_test = csr_matrix((data_test['action'], (data_test['id_enc'], data_test['uid_device_enc'])))

display(data_train.head())
display(data_test.head())

(13351, 12)


,id,uid_device,action,category,geo_address_id,isactive,price,fv_category,fv_geo,ptn_dadd,uid_device_enc,id_enc
0,273726522,00312318-29A0-4E1C-BD78-C0622E14AB66,1,flatSale,1,1,12000000,flatSale,1.0,2022-06-21,0,0
1,274140870,00312318-29A0-4E1C-BD78-C0622E14AB66,1,flatSale,1,1,11499000,flatSale,1.0,2022-06-21,0,1
2,272703932,004639AD-0CE4-4F63-88CD-BAFA8A5BBC70,1,flatSale,1,1,44500000,flatSale,1.0,2022-06-21,1,2
3,274719451,004639AD-0CE4-4F63-88CD-BAFA8A5BBC70,1,flatSale,1,0,15500000,flatSale,1.0,2022-06-21,1,3
4,273874137,004639AD-0CE4-4F63-88CD-BAFA8A5BBC70,1,flatSale,1,1,8500000,flatSale,1.0,2022-06-21,1,4


,id,uid_device,action,category,geo_address_id,isactive,price,fv_category,fv_geo,ptn_dadd,uid_device_enc,id_enc
4450,250969698,12549234,3,flatRent,1,1,45000,flatRent,1.0,2022-06-21,394,4314
4451,205741258,12549234,1,flatRent,1,0,45000,flatRent,1.0,2022-06-21,394,4315
4452,271131332,12555246,1,flatRent,1,1,48000,flatRent,1.0,2022-06-21,395,4316
4453,270853427,12555246,1,flatRent,1,1,52000,flatRent,1.0,2022-06-21,395,4317
4454,272609308,12555246,1,flatRent,1,1,39000,flatRent,1.0,2022-06-21,395,4318


In [ ]:
# check correctness
# users х items
# id - объявление, uid_device - пользователь
display(csr_user_item_train[:10, :10].toarray())
csr_user_item_train.shape

array([[1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 1, 1, 1, 1, 1, 1, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

(395, 4314)

In [ ]:
# make test and train datasets
train_users = set(data_train['uid_device_enc'].values)
train_items = set(data_train['id_enc'].values)


data_test = data_test[(data_test['uid_device_enc'].isin(train_users)) & 
                      (data_test['id_enc'].isin(train_items))]

# make test dict
test_dict = {}
for user_id in data_test['uid_device_enc'].drop_duplicates().values:
    test_dict[user_id] = list(data_test[data_test['uid_device_enc'] == user_id]['id_enc'])

In [ ]:
test_dict

{309: [2609, 2613],
 317: [3451],
 326: [3349],
 340: [3633],
 382: [645],
 391: [3968, 256, 2504],
 394: [439, 1629, 3881, 173, 2323, 714]}

In [ ]:
data_train[data_train['uid_device_enc'] == 394]

,id,uid_device,action,category,geo_address_id,isactive,price,fv_category,fv_geo,ptn_dadd,uid_device_enc,id_enc
4397,258531780,12549234,1,flatRent,1,0,40000,flatRent,1.0,2022-06-21,394,4267
4398,274624926,12549234,1,flatRent,1,1,40000,flatRent,1.0,2022-06-21,394,4099
4399,223292060,12549234,1,flatRent,1,0,35000,flatRent,1.0,2022-06-21,394,4268
4400,274607125,12549234,1,flatRent,1,0,64100,flatRent,1.0,2022-06-21,394,4269
4401,272823340,12549234,1,flatRent,1,0,70000,flatRent,1.0,2022-06-21,394,4270
4402,274349932,12549234,1,flatRent,1,0,40000,flatRent,1.0,2022-06-21,394,4271
4403,274592683,12549234,1,flatRent,1,0,43000,flatRent,1.0,2022-06-21,394,4272
4404,272951025,12549234,1,flatRent,1,0,40000,flatRent,1.0,2022-06-21,394,4273
4405,274434797,12549234,1,flatRent,1,1,43000,flatRent,1.0,2022-06-21,394,4274
4406,257556900,12549234,1,dailyFlatRent,1,1,2500,flatRent,1.0,2022-06-21,394,4275


In [ ]:
# example of dataset
from lightfm.datasets import fetch_movielens
data1 = fetch_movielens(min_rating=5.0)
print(data1['item_feature_labels'].shape)
data1

(1682,)


{'item_feature_labels': array(['Toy Story (1995)', 'GoldenEye (1995)', 'Four Rooms (1995)', ...,
        'Sliding Doors (1998)', 'You So Crazy (1994)',
        'Scream of Stone (Schrei aus Stein) (1991)'], dtype=object),
 'item_features': <1682x1682 sparse matrix of type '<class 'numpy.float32'>'
 	with 1682 stored elements in Compressed Sparse Row format>,
 'item_labels': array(['Toy Story (1995)', 'GoldenEye (1995)', 'Four Rooms (1995)', ...,
        'Sliding Doors (1998)', 'You So Crazy (1994)',
        'Scream of Stone (Schrei aus Stein) (1991)'], dtype=object),
 'test': <943x1682 sparse matrix of type '<class 'numpy.int32'>'
 	with 2153 stored elements in COOrdinate format>,
 'train': <943x1682 sparse matrix of type '<class 'numpy.int32'>'
 	with 19048 stored elements in COOrdinate format>}

In [ ]:
# instantiate and train the model
model = LightFM(loss='warp', no_components=10)
model.fit(csr_user_item_train, epochs=100, num_threads=7, verbose=True) # verbose

Epoch:  24%|██▍       | 24/100 [01:33<04:55,  3.89s/it]


KeyboardInterrupt: ignored

In [ ]:
#predictions = model.predict(np.array(merged['id_enc']), np.array(merged['uid_device_enc']))

all_items = list(train_items)
#all_items = data['id_enc'].values
def get_predictions(user_id, topk=100):
    preds = model.predict(user_id, all_items)
    inds = np.argsort(preds)[::-1]
    return inds[:topk]

preds = {}

for user_id in data_test['uid_device_enc'].drop_duplicates().values:
    pred_user = get_predictions(int(user_id), topk=5)
    preds[user_id] = pred_user.tolist()

In [ ]:
preds[379256]

### Evaluate

In [ ]:
from sklearn.utils import column_or_1d
from sklearn.utils.multiclass import type_of_target

def precision_at_k(y_true, y_score, k):
    """
        INPUT:
        y_true - list of true test values for one user
        y_score - list of predictions values foro one user
        k - level of precision
        OUTPUT:
        ans - precision at k :)
    """
    ans = 0
    counter = 0
    for row in y_true:
        counter += 1
        if counter > k:
            return ans
        if row in y_score:
            ans += 0.2
    return ans

prec_fin = 0
for ind, val in test_dict.items():
    #print(val,  preds[ind])
    prec_fin += precision_at_k(val, preds[ind], 5)
    
prec_fin /= len(test_dict)

In [ ]:
prec_fin

0.02093023255813953

### Polara SVD

In [ ]:
data_model = RecommenderData(data_train, 'id', 'uid_device', 'action')
svd = SVDModel(data_model)
svd.build()
svd.evaluate()

Preparing data...
Users are not uniformly ordered! Unable to split test set reliably.
10086 unique uid_device entities within 20254 testset interactions were filtered. Reason: not in the training data.
1326 unique uid_device entities within 2397 holdout interactions were filtered. Reason: not in the training data.
4692 of 43226 id entities were filtered out from holdout. Reason: incompatible number of items.
2105 id entities were filtered out from holdout. Reason: inconsistent with testset.
1982 id entities were filtered out from testset. Reason: inconsistent with holdout.
Done.
There are 4494129 events in the training and 109287 events in the holdout.
PureSVD training time: 2.189s


[Relevance(precision=nan, recall=0.009378974626442304, fallout=None, specifity=None, miss_rate=0.9906210253735578),
 Ranking(ndcg=0.008195295083381985, ndcl=None, map=0.0038298060690934123, arhr=0.011241120474835371),
 Experience(coverage=0.0006564723562746007),
 Hits(true_positive=1025, false_positive=363265, true_negative=None, false_negative=108262)]

### Polara Random model

In [ ]:
rand = RandomModel(data_model)
rand.build()
rand.evaluate()

[Relevance(precision=-1.4777480048007943, recall=9.150219147748588e-06, fallout=None, specifity=None, miss_rate=0.9999908497808522),
 Ranking(ndcg=3.821454083602119e-07, ndcl=None, map=3.0500730492495306e-06, arhr=9.150219147748588e-06),
 Experience(coverage=0.6285641430458689),
 Hits(true_positive=1, false_positive=364289, true_negative=None, false_negative=109286)]

### Evaluate all (precision at 5)

In [ ]:
svd._user_scores(1)

(array([[ 2.16215671e-05, -1.81520861e-06,  1.05826899e-04, ...,
          6.38535681e-06,  1.15344866e-05, -4.02257841e-05]]),
 (array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0]),
  array([300174,  99932, 111261,  73490, 114706,  33054,  60978, 306119,
         330668, 324725, 109250, 314361, 241708, 317146, 246996, 212861,
         143391, 269214, 219646, 193843,   1904,  23251,  45224, 264195,
         169023, 219653,  55287, 317319, 258526]),
  array([1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 3,
         1, 1, 1, 1, 1, 1, 1], dtype=int64)))

In [ ]:
ans = svd.get_recommendations()

In [ ]:
data_train['uid_device_enc'].drop_duplicates().shape

(379257,)

In [ ]:
preds[379256]

[33220, 60597, 61853, 20162, 60635]

### My SVD

In [ ]:
import scipy.sparse as sp
from scipy.sparse.linalg import svds
from scipy.linalg import sqrtm

def svd(train, k):  
    U, s, V = svds(train.asfptype(), k=k) 
    s = np.diag(s)
    s_root = sqrtm(s)
    Usk = np.dot(U, s_root) 
    skV = np.dot(s_root, V) 
    UsV = np.dot(Usk, skV)
    print("svd готово") 
    return UsV
pred_svd = svd(csr_user_item_train, 10)

In [ ]:
pred_svd.shape

(1289, 16194)

In [ ]:
# sort test dict
test_dict = {}
for user_id in data_test['uid_device_enc'].drop_duplicates().values:
    test_dict[user_id] = list(data_test[data_test['uid_device_enc'] == user_id]['id_enc'])
test_dict

{98: [11673, 6994],
 100: [4646, 2044, 878, 14490],
 152: [12792],
 307: [13571],
 310: [14059],
 547: [13547, 6415, 4728, 4633, 9335, 1709, 15392, 12361, 4707, 12358, 12274],
 648: [15471,
  14227,
  5464,
  7088,
  11728,
  12958,
  15643,
  13071,
  15971,
  7498,
  15642,
  1555],
 973: [15725],
 1048: [7464, 15921],
 1140: [5177, 13944, 5180, 10628, 1452],
 1288: [14505, 10535, 16044, 16053, 2348]}

In [ ]:
# prepare predicted data for evaluation
#diff = pred_svd - csr_user_item_train

#pred_dict = {}
#for i in test_dict.keys():
#    print('f')
#    test_dict[i] = sorted(diff[user_id], reverse=True)

pred_list = []
for i in range(pred_svd.shape[0]):
  pred_list.append(np.argsort(diff[i])[::-1])
pred_list[:5]

[matrix([[  222,     1,   205, ...,  6535, 12421,  7410]]),
 matrix([[   8,    2,    3, ..., 2208, 2174, 2135]]),
 matrix([[ 226,  206,  208, ..., 7945, 2406, 2664]]),
 matrix([[ 255,   12,  908, ..., 9131, 9033, 7230]]),
 matrix([[   13,    14,    15, ..., 10932,  9661,  9694]])]

In [ ]:
from sklearn.utils import column_or_1d
from sklearn.utils.multiclass import type_of_target

def precision_at_k(y_true, y_score, k):
    """
        INPUT:
        y_true - list of true test values for one user
        y_score - list of predictions values foro one user
        k - level of precision
        OUTPUT:
        ans - precision at k :)
    """
    ans = 0
    counter = 0
    for row in y_true:
        counter += 1
        if counter > k:
            return ans
        if row in y_score:
            ans += 1 / k
    return ans

prec_fin = 0
for ind, val in test_dict.items():
    #print(val,  preds[ind])
    prec_fin += precision_at_k(val, pred_list[ind], 5)
    
prec_fin /= len(test_dict)

In [ ]:
prec_fin

0.5818181818181819

In [ ]:
len(set(pred[:5]) & set(true)) / 5